In [ ]:
# treinar com um arquivo e testar com outro
# normalizar/tratar os dados de acordo com o que ta no matlab
# utilizar o preq atual com os pbess e o soc anteriores

In [1]:
"from google.colab import drive drive.mount('/content/drive')"

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.utils import shuffle

import tensorflow as tf

In [3]:
pd.DataFrame([[1, 3, 4], [23, 4543, 22134]], columns = ['amostra 1', 'amostra 2', 'amostra 3'])

,amostra 1,amostra 2,amostra 3
0,1,3,4
1,23,4543,22134


In [33]:
dfTrain = pd.read_csv("Training_dataset6(24_days_subset).csv")
dfTrain.drop(columns=['Unnamed: 0'], inplace=True)

dfTest = pd.read_csv("Testing_dataset6(24_days_subset).csv")
dfTest.drop(columns=['Unnamed: 0'], inplace=True)

In [8]:
dfTest

,Unnamed: 0,P_ESS_Request_W_subset,P_ESS_W_subset,P_L_W_subset,P_PV_W_subset,P_U_W_subset,SOC_subset,Time_Diff_s_subset,TimeStamp_subset
0,0,0,20,21023.8,0.0,21043.8,19,5,['15/04/2020 00:00:00']
1,1,0,20,21006.9,0.0,21026.9,19,5,['15/04/2020 00:00:05']
2,2,0,20,20961.2,0.0,20981.2,19,5,['15/04/2020 00:00:10']
3,3,0,30,20983.1,0.0,21013.1,19,5,['15/04/2020 00:00:15']
4,4,0,30,21068.1,0.0,21098.1,19,5,['15/04/2020 00:00:20']
...,...,...,...,...,...,...,...,...,...
17268,17268,0,20,21170.0,0.0,21190.0,19,5,['15/04/2020 23:59:35']
17269,17269,0,20,21201.2,0.0,21221.2,19,5,['15/04/2020 23:59:40']
17270,17270,0,20,21213.8,0.0,21233.8,19,5,['15/04/2020 23:59:45']
17271,17271,0,20,20611.9,0.0,20631.9,19,5,['15/04/2020 23:59:50']


In [5]:
novo_array = []

for t in range(1, len(dfTrain)):
    novo_array.append([dfTrain["P_ESS_Request_W_subset"][t], dfTrain["P_ESS_W_subset"][t - 1], dfTrain["SOC_subset"][t - 1]])

pd.DataFrame(novo_array, columns = ["P_ESS_Request_W_subset", "P_ESS_W_subset", "SOC_subset"])

dfTrain["P_ESS_Request_W_subset"] = dfTrain["P_ESS_Request_W_subset"] / 13800
dfTrain["P_ESS_W_subset"] = dfTrain["P_ESS_W_subset"] / 13800
dfTrain["SOC_subset"] = dfTrain["SOC_subset"] / 100
dfTrain

,P_ESS_Request_W_subset,P_ESS_W_subset,P_L_W_subset,P_PV_W_subset,P_U_W_subset,SOC_subset,Time_Diff_s_subset,TimeStamp_subset
0,0.0,0.001449,20850.0,0.0,20870.0,0.19,5,['16/04/2020 00:00:00']
1,0.0,0.001449,21141.9,0.0,21161.9,0.19,5,['16/04/2020 00:00:05']
2,0.0,0.001449,21134.4,0.0,21154.4,0.19,5,['16/04/2020 00:00:10']
3,0.0,0.001449,20541.9,0.0,20561.9,0.19,5,['16/04/2020 00:00:15']
4,0.0,0.001449,20669.4,0.0,20689.4,0.19,5,['16/04/2020 00:00:20']
...,...,...,...,...,...,...,...,...
69108,0.0,0.001449,20361.9,0.0,20381.9,0.18,5,['19/04/2020 23:59:37']
69109,0.0,0.001449,21463.8,0.0,21483.8,0.18,5,['19/04/2020 23:59:42']
69110,0.0,0.001449,21444.4,0.0,21464.4,0.18,5,['19/04/2020 23:59:47']
69111,0.0,0.001449,21393.1,0.0,21413.1,0.18,5,['19/04/2020 23:59:52']


In [34]:
def normalizarDados(data):
    data["P_ESS_Request_W_subset"] = data["P_ESS_Request_W_subset"] / 13800
    data["P_ESS_W_subset"] = data["P_ESS_W_subset"] / 13800
    data["SOC_subset"] = data["SOC_subset"] / 100

    novo_array = []

    for t in range(1, len(data)):
        novo_array.append([data["P_ESS_Request_W_subset"][t], data["P_ESS_W_subset"][t - 1], data["SOC_subset"][t - 1]])

    return pd.DataFrame(novo_array, columns = ["P_ESS_Request_W_subset", "P_ESS_W_subset", "SOC_subset"])

In [35]:
novoTreino = normalizarDados(dfTrain)
novoTeste = normalizarDados(dfTest)

In [36]:
novoTreino = shuffle(novoTreino)
novoTreino = pd.DataFrame(novoTreino)

novoTeste = shuffle(novoTeste)
novoTeste = pd.DataFrame(novoTeste)

In [37]:
X_train = novoTreino.iloc[:, :].values
X_test = novoTeste.iloc[:, :].values

Y_train = novoTreino.iloc[:, 1].values
Y_test = novoTeste.iloc[:, 1].values

In [38]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(Y_train.reshape(-1,1))
Y_train =  enc.transform(Y_train.reshape(-1,1)).toarray()

enc.fit(Y_test.reshape(-1,1))
Y_test =  enc.transform(Y_test.reshape(-1,1)).toarray()

In [6]:
sc = StandardScaler()
X_train_Sc = sc.fit_transform(X_train)
X_test_Sc = sc.transform(X_test)

In [30]:
# Inicialização
classifier = tf.keras.models.Sequential()

# Adicionando a camada de entrada e a primeira camada escondida
classifier.add( tf.keras.layers.Dense(input_dim = 3, units = 3, kernel_initializer = 'uniform'))
classifier.add( tf.keras.layers.Dense(units = 15, kernel_initializer = 'uniform'))
classifier.add( tf.keras.layers.Dense( activation = 'sigmoid', units = 10, kernel_initializer = 'uniform'))
classifier.add( tf.keras.layers.Dense( activation = 'sigmoid', units = 5, kernel_initializer = 'uniform'))

# Adicionando a camada de saída
classifier.add( tf.keras.layers.Dense(units = 1, kernel_initializer = 'uniform'))

c:\Users\warma\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [31]:
classifier.compile(optimizer = 'adam', loss = 'mse', metrics = ['mse'])

In [39]:
classifier.fit(X_train, Y_train, epochs = 600)

Epoch 1/600
2160/2160 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 5.0277e-04 - mse: 5.0277e-04
Epoch 2/600
2160/2160 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 5.0277e-04 - mse: 5.0277e-04
Epoch 3/600
2160/2160 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 5.0277e-04 - mse: 5.0277e-04
Epoch 4/600
2160/2160 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 5.0277e-04 - mse: 5.0277e-04
Epoch 5/600
2160/2160 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 5.0277e-04 - mse: 5.0277e-04
Epoch 6/600
2160/2160 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 5.0277e-04 - mse: 5.0277e-04
Epoch 7/600
2160/2160 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 5.0277e-04 - mse: 5.0277e-04
Epoch 8/600
2160/2160 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 5.0277e-04 - mse: 5.0277e-04
Epoch 9/600
2160/2160 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 5.0277e-04 - mse: 5.0277e-04
Epoch 10/600
2160/2160 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 5.0277e-04 - mse: 5.0277e-04
Epoch 11/600
2160/2160 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 5.0277e-04 - ms

In [10]:
y_pred = classifier.predict(X_test)

432/432 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [11]:
print(Y_test[0:35])
print(y_pred[0:35])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461333e-05]
 [3.0461333e-05]
 [3.0461331e-05]
 [3.0461331e-05]
 [3.0461331e-05]]


In [14]:
print(classifier.metrics)

[<Mean name=loss>, <CompileMetrics name=compile_metrics>]
